# Benchmarking tflite models  

This notebook parses the benchmarking.txt files generated by: 

 [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)  

 TensorFlow Lite benchmark tools currently measure and calculate statistics for the following important performance metrics:

- Initialization time
- Inference time of warmup state
- Inference time of steady state
- Memory usage during initialization time
- Overall memory usage

In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths

In [2]:
import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd


# enable plotly in VS Studio Code
#import plotly.io as pio
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "plotly_mimetype+notebook"

import wandb

In [3]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
models_dir = initialize()

In [5]:
global model_name
#model_name = "efficientNetB0_1_96_c1_o3_keras"
#model_name = "mobilenetv2_0.1_96_c1_o3_l1"
#model_name = "mobilenetv1_0.1_96_c3_o3_l2"
model_name = "shufflenetv2tiny_0.1_96_c3_o3_f4l1024"

models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [6]:
models_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_benchmark.txt")
models_benchmark_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv2tiny_0.1_96_c3_o3_f4l1024/shufflenetv2tiny_0.1_96_c3_o3_f4l1024_benchmark.txt')

# Helper functions

In [7]:
def clean_model_summary(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [8]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [9]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [10]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

if models_benchmark_path.is_file():
    

In [12]:
lines = clean_model_summary(models_benchmark_path)
lines

['STARTING!',
 'Log parameter values verbosely: [0]',
 'Num threads: [1]',
 'Graph: [/mnt/i/tinyml/tiny_cnn/models/shufflenetv2tiny_0.1_96_c3_o3_f4l1024/shufflenetv2tiny_0.1_96_c3_o3_f4l1024_INT8.tflite]',
 'Enable op profiling: [1]',
 '#threads used for CPU inference: [1]',
 'Loaded model /mnt/i/tinyml/tiny_cnn/models/shufflenetv2tiny_0.1_96_c3_o3_f4l1024/shufflenetv2tiny_0.1_96_c3_o3_f4l1024_INT8.tflite',
 'The input model file size (MB): 0.26636',
 'Initialized session in 164.461ms.',
 'Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.',
 'count=1167 first=12212 curr=465 min=357 max=12212 avg=426.317 std=356',
 'Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.',
 'count=1563 first=480 curr=583 min=384 max=4892 avg=550.678 std=289',
 'Inference timings in us: Init: 164461, First inference: 12212, Warmup (avg): 426.317, Inference (avg): 550.678',
 'Note: as the benchma

In [13]:
def get_profiling_lines(lines):
    #print(lines)
    # find section split
    for i, line in enumerate(lines):
        #print(f"{i} : {line}")
    #if line.str.contains('= Run Order ='):
        if "Operator-wise Profiling " in line:
            split_line = i
            #print(f"Splitting text at line {split_line}")
            model_profiling = lines[:split_line]
            operator_profiling = lines[split_line+1:]

        else:
            pass
    if i==0:     
        print("WARNING: Could not find the text: 'Operator-wise Profiling Info for Regular Benchmark Runs:'")
        return (None, None)
    else:
        return model_profiling, operator_profiling


In [14]:
model_profiling, operator_profiling = get_profiling_lines(lines)

In [58]:
def get_index_of_model_profiling_lines(lines):
    model_lines_dict = {}
    for i, line in enumerate(lines):
        if "= Run Order =" in line:
            model_lines_dict["run_order"] = i
        elif "= Top by Computation Time =" in line:
            model_lines_dict["top_by_computation_time"] = i
        elif "The input model file size" in line:
            model_lines_dict["model_file_size"] = i
        elif "Initialized session in" in line:
            model_lines_dict["initialization_ms"] = i
        elif "Inference timings in us:" in line:
            model_lines_dict["inference_timings"] = i

    #print(model_lines_dict)
    return model_lines_dict


In [59]:
model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

In [61]:
def get_model_profiling_stats(model_profiling):
    model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

    model_stats = {}

    # extract model size
    model_size_string = model_profiling[model_lines_dict["model_file_size"]]
    model_size_string
    model_size= float(model_size_string.split(":")[-1].strip())
    model_stats["model_size_MB"] = model_size

    # extract inference times
    inference_string = model_profiling[model_lines_dict["inference_timings"]]
    #print(inference_string)
    inference_strings = inference_string.split(",")
    for item in inference_strings:
        item_time = item.split()[-1]
        #print(item_time)

    model_stats["init_us"] = int(inference_strings[0].split()[-1].strip())
    model_stats["first_inference_us"] = int(inference_strings[1].split()[-1].strip())
    model_stats["warmup_avg_us"] = float(inference_strings[2].split()[-1].strip())
    model_stats["inference_avg_us"] = float(inference_strings[3].split()[-1].strip())

    # model initialization timings
    init_string = model_profiling[model_lines_dict["initialization_ms"]]
    model_stats["initialization_ms"] = float(init_string.split()[-1].replace("ms.",""))


    # initialization run order
    init_run_order = model_profiling[model_lines_dict["run_order"]: model_lines_dict["top_by_computation_time"]]
    init_run_order = remove_tabs(init_run_order)

    model_stats["modify_graph_with_delegate_ms_first"] = float(init_run_order[2][1].strip())
    model_stats["modify_graph_with_delegate_ms_avg"] = float(init_run_order[2][2].strip())
    model_stats["modify_graph_with_delegate_ms_%"] = float(init_run_order[2][3].strip().replace("%", ""))
    model_stats["modify_graph_with_delegate_mem_KB"] = float(init_run_order[2][5].strip())

    model_stats["allocate_tensors_ms_first"] = float(init_run_order[3][1].strip())
    model_stats["allocate_tensors_ms_avg"] = float(init_run_order[3][2].strip())
    model_stats["allocate_tensors_ms_%"] = float(init_run_order[3][3].strip().replace("%", ""))

    return model_stats

In [23]:
model_stats = get_model_profiling_stats(model_profiling)
model_stats

{'model_size_MB': 0.26636,
 'init_us': 164461,
 'first_inference_us': 12212,
 'warmup_avg_us': 426.317,
 'inference_avg_us': 550.678,
 'initialization_ms': 164.461,
 'modify_graph_with_delegate_ms_first': 8.256,
 'modify_graph_with_delegate_ms_avg': 8.256,
 'modify_graph_with_delegate_ms_%': 98.426,
 'modify_graph_with_delegate_mem_KB': 1096.0,
 'allocate_tensors_ms_first': 0.132,
 'allocate_tensors_ms_avg': 0.132,
 'allocate_tensors_ms_%': 1.574}

In [24]:
def get_profiling_stats_cpu(filepath):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        dict: dictionary with extracted model summary profiling information 
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    profiling_stats = get_model_profiling_stats(model_profiling)
    return profiling_stats


In [25]:
prof_stats = get_profiling_stats_cpu(models_benchmark_path)
prof_stats

{'model_size_MB': 0.26636,
 'init_us': 164461,
 'first_inference_us': 12212,
 'warmup_avg_us': 426.317,
 'inference_avg_us': 550.678,
 'initialization_ms': 164.461,
 'modify_graph_with_delegate_ms_first': 8.256,
 'modify_graph_with_delegate_ms_avg': 8.256,
 'modify_graph_with_delegate_ms_%': 98.426,
 'modify_graph_with_delegate_mem_KB': 1096.0,
 'allocate_tensors_ms_first': 0.132,
 'allocate_tensors_ms_avg': 0.132,
 'allocate_tensors_ms_%': 1.574}

# Operator profiling

In [26]:
def get_operator_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    # try:
    df["%"] = df["%"].apply(string_percent_to_float)
    # except:
    #     df["avg %"] = df["avg %"].apply(string_percent_to_float)
    # try:
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    # except:
    #     df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)#
    #df.Name = name

    return df

In [27]:
def get_node_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    df["avg %"] = df["avg %"].apply(string_percent_to_float)
    df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)
    df["count"] = df["count"].map(int)
    df.Name = name

    return df

In [28]:
def get_index_of_operator_profiling_lines(lines):
    operator_lines_dict = {}
    for i, line in enumerate(lines):
        #if line.str.contains('= Run Order ='):
        if "= Run Order =" in line:
            operator_lines_dict["run_order"] = i
        elif "= Top by Computation Time =" in line:
            operator_lines_dict["top_by_computation_time"] = i
        elif "= Summary by node type =" in line:
            operator_lines_dict["summary_by_node_type"] = i

    # print(operator_lines_dict)
    return operator_lines_dict

In [29]:
# operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)
# operator_lines_dict

In [62]:
def get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name):
    summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
    summary_by_node_type = remove_tabs(summary_by_node_type)
    summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
    return summary_by_node_type_df

In [63]:
def get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name):

    top_by_comp_time = operator_profiling[
        operator_lines_dict["top_by_computation_time"]+ 1 : operator_lines_dict["summary_by_node_type"]- 1]
    top_by_comp_time_clean = remove_tabs(top_by_comp_time)

    df = pd.DataFrame(top_by_comp_time_clean)

    df.rename(columns=df.iloc[0, :], inplace=True)
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)

    df["%"] = df["%"].apply(string_percent_to_float)
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)

    return df

In [64]:
def get_run_order_df(operator_lines_dict, operator_profiling, model_name):
    run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
    run_order = remove_tabs(run_order)
    df = get_operator_df(run_order, name= f"Run order - {model_name}")
    return df

In [65]:
def get_profiling_dataframes_cpu(filepath, model_name):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        list(pd.DataFrame, pd.DataFrame, pd.DataFrame): tuple containing the following dataframes:

            operator_run_order_df, operator_by_comp_time_df, summary_node_type_df
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)

    operator_run_order_df = get_run_order_df(operator_lines_dict, operator_profiling, model_name)
    operator_by_comp_time_df = get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name)
    summary_node_type_df = get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name)

    return operator_run_order_df, operator_by_comp_time_df, summary_node_type_df

In [68]:
operator_run_order_df, operator_by_comp_time_df, summary_node_type_df = get_profiling_dataframes_cpu(models_benchmark_path, model_name)

In [69]:
operator_run_order_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,QUANTIZE,0.000,0.001,0.190,0.190,0.0,1,[StatefulPartitionedCall:0]:138
2,SOFTMAX,0.001,0.001,0.243,0.433,0.0,1,[StatefulPartitionedCall:01]:137
3,"Fully Connected (NC, QS8) GEMM",0.001,0.001,0.127,0.560,0.0,1,"Delegate/Fully Connected (NC, QS8) GEMM:2"
4,"Global Average Pooling (NWC, QS8)",0.001,0.001,0.325,0.885,0.0,1,"Delegate/Global Average Pooling (NWC, QS8):1"
5,RESHAPE,0.001,0.001,0.126,1.011,0.0,1,[shufflenetv2tiny/tf.reshape_31/Reshape]:133
6,RESHAPE,0.001,0.001,0.137,1.148,0.0,1,[shufflenetv2tiny/tf.reshape_30/Reshape]:131
7,RESHAPE,0.000,0.001,0.133,1.281,0.0,1,[shufflenetv2tiny/tf.reshape_29/Reshape]:125
8,RESHAPE,0.000,0.001,0.149,1.429,0.0,1,[shufflenetv2tiny/tf.reshape_28/Reshape]:123
9,RESHAPE,0.001,0.001,0.132,1.562,0.0,1,[shufflenetv2tiny/tf.reshape_27/Reshape]:117
10,RESHAPE,0.001,0.001,0.142,1.703,0.0,1,[shufflenetv2tiny/tf.reshape_26/Reshape]:115


In [70]:
operator_by_comp_time_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,QUANTIZE,0.110,0.125,30.023,30.023,0.0,1,[tfl.quantize]:0
2,"Convolution (NHWC, QC8) IGEMM",0.047,0.056,13.441,43.464,0.0,1,"Delegate/Convolution (NHWC, QC8) IGEMM:0"
3,"Convolution (NHWC, QC8) GEMM",0.004,0.012,8.694,52.158,0.0,3,"Delegate/Convolution (NHWC, QC8) GEMM:0"
4,"Max Pooling (NHWC, S8)",0.005,0.006,1.372,53.530,0.0,1,"Delegate/Max Pooling (NHWC, S8):1"
5,"Convolution (NHWC, QC8) DWConv",0.004,0.004,1.025,54.555,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:5"
6,"Convolution (NHWC, QC8) GEMM",0.007,0.004,2.649,57.204,0.0,3,"Delegate/Convolution (NHWC, QC8) GEMM:2"
7,"Transpose (ND, X8)",0.005,0.003,12.252,69.456,0.0,16,"Delegate/Transpose (ND, X8):0"
8,"Convolution (NHWC, QC8) GEMM",0.002,0.003,0.642,70.098,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:6"
9,"Convolution (NHWC, QC8) DWConv",0.004,0.002,6.864,76.962,0.0,13,"Delegate/Convolution (NHWC, QC8) DWConv:2"
10,"Convolution (NHWC, QC8) DWConv",0.003,0.002,1.420,78.382,0.0,3,"Delegate/Convolution (NHWC, QC8) DWConv:3"


In [71]:
summary_node_type_df

,Node type,count,avg ms,avg %,cdf %,mem KB,times called
1,QUANTIZE,2,0.124,32.208,32.208,0.0,2
2,"Convolution (NHWC, QC8) GEMM",6,0.094,24.416,56.623,0.0,36
3,"Convolution (NHWC, QC8) IGEMM",1,0.055,14.286,70.909,0.0,1
4,"Transpose (ND, X8)",1,0.050,12.987,83.896,0.0,16
5,"Convolution (NHWC, QC8) DWConv",4,0.039,10.130,94.026,0.0,19
6,"Copy (NC, X8)",4,0.016,4.156,98.182,0.0,29
7,"Max Pooling (NHWC, S8)",1,0.005,1.299,99.481,0.0,1
8,SOFTMAX,1,0.001,0.260,99.740,0.0,1
9,"Global Average Pooling (NWC, QS8)",1,0.001,0.260,100.000,0.0,1
10,RESHAPE,32,0.000,0.000,100.000,0.0,32


In [38]:
# operator_profiling = lines[split_line+1:]
# operator_profiling

In [39]:
# operator_lines_dict = {}
# for i, line in enumerate(operator_profiling):
#     #if line.str.contains('= Run Order ='):
#     if "= Run Order =" in line:
#         operator_lines_dict["run_order"] = i
#     elif "= Top by Computation Time =" in line:
#         operator_lines_dict["top_by_computation_time"] = i
#     elif "= Summary by node type =" in line:
#         operator_lines_dict["summary_by_node_type"] = i

# operator_lines_dict

In [40]:
# summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
# summary_by_node_type = remove_tabs(summary_by_node_type)
# summary_by_node_type

In [41]:
# summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
# summary_by_node_type_df

In [42]:
#summary_by_node_type_df.Name

In [43]:
#summary_by_node_type_df.info

In [44]:
# top_by_comp_time = operator_profiling[operator_lines_dict["top_by_computation_time"] +1: operator_lines_dict["summary_by_node_type"]-1]
# top_by_comp_time = remove_tabs(top_by_comp_time)
# top_by_comp_time

In [45]:
# df_operator_top_by_comp_time = get_operator_df(top_by_comp_time, name=f"Top by compute time - {model_name}")
# df_operator_top_by_comp_time

In [46]:
# run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
# run_order = remove_tabs(run_order)
# #run_order

In [47]:
# df_operator_run_order = get_operator_df(run_order, name= f"Run order - {model_name}")
# #df_operator_run_order.style.set_properties(**{'text-align': 'left'})
# df_operator_run_order.head(20)

In [48]:
# df_operator_run_order.info()

# Peak memory usage

In [49]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")
models_peak_memory_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv2tiny_0.1_96_c3_o3_f4l1024/shufflenetv2tiny_0.1_96_c3_o3_f4l1024_peak_memory')

In [50]:
# ! ./benchmarking/linux_x86-64_benchmark_model \
#     --graph=$models_tflite_path \
#     --num_threads=1 \
#     --enable_op_profiling=true \
#     | tee $models_benchmark_path

In [51]:
models_tflite_opt_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv2tiny_0.1_96_c3_o3_f4l1024/shufflenetv2tiny_0.1_96_c3_o3_f4l1024_INT8.tflite')

In [52]:
# ! python i:\\tinyml\\tflite-tools\\tflite_tools.py \
#     -i $models_tflite_opt_path \
#     --csv $models_peak_memory_path 
 

In [53]:
! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path

Traceback (most recent call last):
  File "i:\tinyml\tflite-tools\tflite_tools.py", line 49, in <module>
    main()
  File "i:\tinyml\tflite-tools\tflite_tools.py", line 23, in main
    model = TFLiteModel.load_from_file(args.input_path)
  File "i:\tinyml\tflite-tools\tflite_tools\tflite_model.py", line 119, in load_from_file
    return cls(bytearray(f.read()))
  File "i:\tinyml\tflite-tools\tflite_tools\tflite_model.py", line 114, in __init__
    self._build_graph()
  File "i:\tinyml\tflite-tools\tflite_tools\tflite_model.py", line 174, in _build_graph
    assert op.OutputsLength() <= 1
AssertionError


In [54]:
%%capture peak_memory_str

! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path

In [55]:
x = peak_memory_str.stdout
peak_memory_lines = x.split("\n")
peak_memory_lines

['']

In [56]:
peak_mem_lines_dict = {}
for i, line in enumerate(peak_memory_lines):
    #if line.str.contains('= Run Order ='):
    if "|  Id | " in line:
        peak_mem_lines_dict["header"] = i
    elif "Current peak memory usage:" in line:
        peak_mem_lines_dict["peak_memory"] = i
    elif "Operator execution schedule:" in line:
        peak_mem_lines_dict["operator_schedule"] = i

peak_mem_lines_dict

{}

In [57]:
operator_schedule = peak_memory_lines[peak_mem_lines_dict["operator_schedule"]: peak_mem_lines_dict["peak_memory"]]
operator_schedule

KeyError: 'operator_schedule'

In [ ]:
operator_lines =[]
for line in operator_schedule:
    if line.startswith("|"):
        operator_lines.append(line.strip("|"))
#operator_lines

operator_data =[]
for row in operator_lines:
    operator_data_dict = {}
    data =row.split("|")
    #print(data)
    operator_data_dict["operator"] = data[0].strip()
    operator_data_dict["tensor_ids_in_memory"] = data[1].strip()
    operator_data_dict["peak_memory_kb"] = data[2].strip().replace(",", "")
    operator_data.append(operator_data_dict)

operator_data

In [ ]:
peak_memory_df = pd.DataFrame.from_dict(operator_data)
peak_memory_df = peak_memory_df.drop([0])
peak_memory_df["peak_memory_kb"] = peak_memory_df["peak_memory_kb"].map(float)/1000.0
peak_memory_df.Name = f"Peak memory - {model_name}"
peak_memory_df

In [ ]:
models_peak_memory_df_path = models_dir.joinpath(model_name, "peak_memory_df")
models_peak_memory_df_path

In [ ]:
peak_memory_df.to_pickle(models_peak_memory_df_path)

In [ ]:
peak_memory_df["peak_memory_kb"].max()

In [ ]:
tensor_information = peak_memory_lines[:peak_mem_lines_dict["operator_schedule"]]
tensor_information

In [ ]:
data_lines =[]
for line in tensor_information:
    if line.startswith("|"):
        data_lines.append(line.strip("|"))
#data_lines

split_data =[]
for row in data_lines:
    line_dict = {}
    data =row.split("|")
    #print(data)
    line_dict["tensor_id"] = data[0].strip()
    line_dict["tensor"] = data[1].strip()
    line_dict["shape"] = data[2].strip()
    line_dict["size_in_RAM_kb"] = data[3].strip().replace(",", "")
    split_data.append(line_dict)

split_data


In [ ]:
tensor_df = pd.DataFrame.from_dict(split_data)
tensor_df = tensor_df.drop([0])
tensor_df["size_in_RAM_kb"] = tensor_df["size_in_RAM_kb"].map(float)/1000.0
tensor_df.Name = f"Tensor information - {model_name}"
tensor_df

In [ ]:
tensor_df.Name

In [ ]:
tensor_info_df_path = models_dir.joinpath(model_name, "tensor_info_df")
tensor_info_df_path

In [ ]:
tensor_df.to_pickle(tensor_info_df_path)

## Parse peak memory

In [ ]:
peak_memory_line = peak_memory_lines[peak_mem_lines_dict["peak_memory"]]
peak_memory_line

In [ ]:
peak_memory_kb = float(peak_memory_line.split(":")[-1].strip().split()[0].replace(",", ".") )
peak_memory_kb

In [ ]:
model_stats["peak_memory_kb"] = peak_memory_kb

In [ ]:
model_stats

# Logging to wandb

In [ ]:
# Generate run ids
id = wandb.util.generate_id()

PROJECT = model_name.split("_")[0]

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        name = model_name,
        id = id, 
        resume="allow",
        sync_tensorboard=True
        )
# Specify the configuration variables
config = wandb.config
config.architecture = model_name

# # Create a table
operator_top_by_time_table = wandb.Table(dataframe=df_operator_top_by_comp_time)
# operator_top_by_time_artifact = wandb.Artifact(name=f'{model_name}_operator_top_by_time', type='dataframe')
# operator_top_by_time_artifact.add(operator_top_by_time_table, "Profiling operators - top by time")
# run.log_artifact(operator_top_by_time_artifact)

summary_by_node_type_table = wandb.Table(dataframe=summary_by_node_type_df)
# summary_by_node_type_artifact = wandb.Artifact(name=f'{model_name}_summary_by_node_type', type='dataframe')
# summary_by_node_type_artifact.add(summary_by_node_type_table, "Node type summary")
# run.log_artifact(summary_by_node_type_artifact)

operator_run_order_table = wandb.Table(dataframe=df_operator_run_order)
# operator_run_order_artifact = wandb.Artifact(name=f'{model_name}_operator_run_order', type='dataframe')
# operator_run_order_artifact.add(operator_run_order_table, "Run order")
# run.log_artifact(operator_run_order_artifact)


run.log({"Profiling operators - top by time": operator_top_by_time_table})
run.log({"Node type summary" : summary_by_node_type_table})
run.log({"Run order" : operator_run_order_table})
# run.log({"Chart_table": table})

# table = wandb.Table(columns = ["plotly_figure"])

# # Create path for Plotly figure
# path_to_plotly_html = "./plotly_figure.html"

# # Write Plotly figure to HTML
# fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# # Add Plotly figure as HTML file into Table
# table.add_data(wandb.Html(path_to_plotly_html))

# # Log Table
# run.log({"Chart_table": table})

wandb.log(model_stats)

# #wandb.log({"Peak activations chart": fig})

wandb.finish()